Spelling error ||
variable not in the function ||
return staement incorrect || 
return statement should be consistent while calling and while defining the func

In [1]:
import pandas as pd
import numpy as np 
import datetime as dt

In [2]:
start_time = "09:25:00"
end_time = "09:29:00"

trade_start_time = '09:30:00'
trade_end_time = '10:30:00'

tgt_pts = 100

# This date is an outlier. On this day Trading happens at a different time-frame.
muhurat_trading_day = '04-11-2021'

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/Sreemanto/SK__007/master/AlgoTrading%20Workshop/BANK%20NIFTY%20FUT%202021.csv')

In [11]:
#filtering data for the month of jan 
#df = df[df['month']=='JAN']


#converting date column to datetime format 
df['date'] = pd.to_datetime(df['date'])

#creating separate columns for day and time 
df['Day']= df['date'].dt.date
df['time'] = df['date'].dt.time

Days = df.Day.unique().tolist()

#Days.remove(muhurat_trading_day)

##### 1. getting highlow
2. Checking for high breach or low breach
3. Checking for entry index
3. Checking for profit or loss

In [12]:
def get_high_low(d1,start_time,end_time):
    '''
    This function is created to find third candle high and low

    '''
    
    df_candle = d1.set_index('date')
    df_candle = df_candle.between_time(start_time,end_time)
    
    
    thrd_cndl_high = df_candle['high'].max()
    thrd_cndl_low = df_candle['low'].min()
    
    print("Candle High = ",thrd_cndl_high)
    print("Candle Low = ",thrd_cndl_low)   
    
    return thrd_cndl_high,thrd_cndl_low

In [13]:
def index_breach(d1,thrd_cndl_high,thrd_cndl_low):
    '''
    This function fetches the index for pionts of high breach or low breach 

    '''
    
     #filtering from 9:30 to 10:30 check if trade should be done
    df_trade= d1.set_index('date')
    df_trade = df_trade.between_time(trade_start_time,trade_end_time)
    
    #reseting the index because we don't want date in the index while calc entry_index
    df_trade.reset_index(inplace=True)
    print(df_trade.shape)
    
    df_trade['high_breach'] =  np.where(df_trade.high > thrd_cndl_high, 1, 0)
    df_trade['low_breach'] = np.where(df_trade.low < thrd_cndl_low, 1, 0)
    
    #finding point of entry index.
    #checkinh if high breach has happened. if yes then note down the index
    
    #checking for high breach
    if df_trade[df_trade['high_breach']==1].shape[0] > 0:
        index_high_breach = df_trade[df_trade['high_breach']==1].head(1).index[0]
    else:
        index_high_breach = -1
        
    #checking for low breach    
    if df_trade[df_trade['low_breach']==1].shape[0] > 0:
        index_low_breach = df_trade[df_trade['low_breach']==1].head(1).index[0]
    else:
        index_low_breach = -1     
    
    print('index high breach:',index_high_breach)
    print('index low breach:',index_low_breach)
    
    return index_high_breach,index_low_breach
    

In [14]:
def position_price(index_high_breach,index_low_breach,tgt_pts):
    '''
    This function fetches the postition of the trande and the entry price 
    
    '''
    
     #checking which happened first ( incase both happened)    
    if index_high_breach!=-1 and index_low_breach!=-1:
        entry_index = min(index_high_breach,index_low_breach)
    elif index_high_breach==-1 and index_low_breach!=-1:
        entry_index = index_low_breach
    elif index_high_breach!=-1 and index_low_breach==-1:
        entry_index = index_high_breach
    else:
        print('No Trade')
        entry_index = -1
        
    if entry_index!=-1:
        if entry_index == index_low_breach:
            position = "Short"
            entry_price = thrd_cndl_low 
            stop_loss = thrd_cndl_high
            target = entry_price - tgt_pts
        else:
            position = "Long"
            entry_price = thrd_cndl_high
            stop_loss = thrd_cndl_low 
            target = entry_price + tgt_pts
        
        print("Position - ",position)
        print("Entry Price - ",entry_price)
        print("Target - ",target)
        print("Stop Loss - ",stop_loss)
        
    else:
        position = "NA"
        entry_price = np.nan
        stop_loss = np.nan 
        target = np.nan
        print("No position as no trade")
        print("Position - ",position)
        print("Entry Price - ",entry_price)
    
    return position,entry_price,target,stop_loss,entry_index
    

In [15]:
def trade_check(targert,stop_loss,position):
    '''
    This function fetches the result and exit price for a trade in the long position 
    
    '''
    if position == "Long":
        df_trade_check['target'] = np.where(df_trade_check.high>target,1,0)
        df_trade_check['stop_loss'] = np.where(df_trade_check.low<stop_loss,1,0)          
    else:        
        df_trade_check['target'] = np.where(df_trade_check.low<target,1,0)
        df_trade_check['stop_loss'] = np.where(df_trade_check.high>stop_loss,1,0)            


    # we are checking for target
    if df_trade_check[df_trade_check['target']==1].shape[0]>0:
        tgt_index = df_trade_check[df_trade_check['target']==1].head(1).index[0]
    else:
        tgt_index = -1

    # we are checking for stop loss
    if df_trade_check[df_trade_check['stop_loss']==1].shape[0]>0:
        sl_index = df_trade_check[df_trade_check['stop_loss']==1].head(1).index[0]
    else:
        sl_index = -1

    #checking if both sl and target was breached
    if tgt_index!=-1 and sl_index!=-1:
        if tgt_index < sl_index:
            exit_index = tgt_index
        else:
            exit_index = sl_index
            
    # checking only for sl 
    elif tgt_index==-1 and sl_index!=-1:
        exit_index = sl_index
    # checking for target
    elif tgt_index!=-1 and sl_index==-1:
        exit_index = tgt_index
    else:
        print('No Trade')


    if exit_index == tgt_index:
        result = "Profit"
        exit_price = target
    else:
        result = "Loss"
        exit_price = stop_loss

        
    print("Result - ",result)
    print("Exit Price - ",exit_price)

    return result, exit_price 

In [16]:
rpt_df = pd.DataFrame()
col_list = ['Date','third_candle_high','third_candle_low','position',
                  'entry_price','target','stop_loss','exit_price','result']

for day in Days:
    
    print('The day is:',day)
    
    #filtering for one day 
    d1 = df[df['Day']==day]
    
    #fetching 3rd cndl low and high
    thrd_cndl_high,thrd_cndl_low = get_high_low(d1,start_time,end_time)
    
    #fetching index of high or low breach 
    index_high_breach,index_low_breach = index_breach(d1,thrd_cndl_high,thrd_cndl_low)
    
    #fetching trade position and entry price 
    
    position,entry_price,target,stop_loss,entry_index = position_price(index_high_breach,index_low_breach,tgt_pts)

        
    if position!="NA":
    
        #creating a df to check the trade 
        df_trade_check = d1[entry_index:]
    
    
        result, exit_price = trade_check(target,stop_loss,position)
            
    else:
        result = "NA"
        exit_price = np.nan
        print("Result - ",result)
        print("Exit Price - ",exit_price)
        
    print("-------------------------------")

    print("-------------------------------")
        
    rpt_temp = pd.DataFrame([day,thrd_cndl_high,thrd_cndl_low,position,
                                   entry_price,target,stop_loss,exit_price,result]).T
    
    rpt_df = pd.concat([rpt_temp,rpt_df])
    
rpt_df.columns = col_list
rpt_df['net_points'] = np.where(rpt_df['position']=='Long',rpt_df['exit_price']-rpt_df['entry_price'],
                                np.where(rpt_df['position']=='Short',rpt_df['entry_price']-rpt_df['exit_price'],0))

rpt_df['amount'] = rpt_df['net_points']*50
rpt_df

The day is: 2021-04-01
Candle High =  31599.0
Candle Low =  31430.0
(61, 11)
index high breach: -1
index low breach: -1
No Trade
No position as no trade
Position -  NA
Entry Price -  nan
Result -  NA
Exit Price -  nan
-------------------------------
-------------------------------
The day is: 2021-05-01
Candle High =  31205.0
Candle Low =  31075.0
(61, 11)
index high breach: 1
index low breach: -1
Position -  Long
Entry Price -  31205.0
Target -  31305.0
Stop Loss -  31075.0


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['target'] = np.where(df_trade_check.high>target,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['stop_loss'] = np.where(df_trade_check.low<stop_loss,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

Result -  Loss
Exit Price -  31075.0
-------------------------------
-------------------------------
The day is: 2021-06-01
Candle High =  31929.0
Candle Low =  31880.0
(61, 11)
index high breach: 2
index low breach: 0
Position -  Short
Entry Price -  31880.0
Target -  31780.0
Stop Loss -  31929.0
Result -  Loss
Exit Price -  31929.0
-------------------------------
-------------------------------
The day is: 2021-07-01
Candle High =  32214.55
Candle Low =  32166.05
(61, 11)
index high breach: 4
index low breach: 2
Position -  Short
Entry Price -  32166.05
Target -  32066.05
Stop Loss -  32214.55
Result -  Loss
Exit Price -  32214.55
-------------------------------
-------------------------------
The day is: 2021-08-01
Candle High =  32227.45
Candle Low =  32170.5
(61, 11)
index high breach: 0
index low breach: 11
Position -  Long
Entry Price -  32227.45
Target -  32327.45
Stop Loss -  32170.5
Result -  Profit
Exit Price -  32327.45
-------------------------------
----------------------

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['target'] = np.where(df_trade_check.high>target,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['stop_loss'] = np.where(df_trade_check.low<stop_loss,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

Result -  Loss
Exit Price -  31850.0
-------------------------------
-------------------------------
The day is: 2021-01-13
Candle High =  32592.0
Candle Low =  32552.0
(61, 11)
index high breach: 0
index low breach: -1
Position -  Long
Entry Price -  32592.0
Target -  32692.0
Stop Loss -  32552.0
Result -  Loss
Exit Price -  32552.0
-------------------------------
-------------------------------
The day is: 2021-01-14
Candle High =  32800.7
Candle Low =  32720.0
(61, 11)
index high breach: -1
index low breach: 0
Position -  Short
Entry Price -  32720.0
Target -  32620.0
Stop Loss -  32800.7
Result -  Profit
Exit Price -  32620.0
-------------------------------
-------------------------------
The day is: 2021-01-15
Candle High =  32578.5
Candle Low =  32516.15
(61, 11)
index high breach: -1
index low breach: 0
Position -  Short
Entry Price -  32516.15
Target -  32416.15
Stop Loss -  32578.5
Result -  Loss
Exit Price -  32578.5
-------------------------------
---------------------------

...

Result -  Loss
Exit Price -  35310.65
-------------------------------
-------------------------------
The day is: 2021-12-24
Candle High =  35194.2
Candle Low =  35056.0
(61, 11)
index high breach: -1
index low breach: 0
Position -  Short
Entry Price -  35056.0
Target -  34956.0
Stop Loss -  35194.2
Result -  Loss
Exit Price -  35194.2
-------------------------------
-------------------------------
The day is: 2021-12-27
Candle High =  34614.0
Candle Low =  34375.85
(61, 11)
index high breach: 0
index low breach: -1
Position -  Long
Entry Price -  34614.0
Target -  34714.0
Stop Loss -  34375.85
Result -  Loss
Exit Price -  34375.85
-------------------------------
-------------------------------
The day is: 2021-12-28
Candle High =  35319.95
Candle Low =  35220.0
(61, 11)
index high breach: 5
index low breach: 0
Position -  Short
Entry Price -  35220.0
Target -  35120.0
Stop Loss -  35319.95
Result -  Loss
Exit Price -  35319.95
-------------------------------
--------------------------

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['target'] = np.where(df_trade_check.low<target,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['stop_loss'] = np.where(df_trade_check.high>stop_loss,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

Result -  Loss
Exit Price -  35238.65
-------------------------------
-------------------------------
The day is: 2021-12-30
Candle High =  34963.9
Candle Low =  34892.8
(122, 11)
index high breach: 0
index low breach: -1
Position -  Long
Entry Price -  34963.9
Target -  35063.9
Stop Loss -  34892.8
Result -  Loss
Exit Price -  34892.8
-------------------------------
-------------------------------


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['target'] = np.where(df_trade_check.high>target,1,0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19980\2423779476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade_check['stop_loss'] = np.where(df_trade_check.low<stop_loss,1,0)


,Date,third_candle_high,third_candle_low,position,entry_price,target,stop_loss,exit_price,result,net_points,amount
0,2021-12-30,34963.9,34892.8,Long,34963.9,35063.9,34892.8,34892.8,Loss,-71.1,-3555.0
0,2021-12-29,35315.65,35238.65,Long,35315.65,35415.65,35238.65,35238.65,Loss,-77.0,-3850.0
0,2021-12-28,35319.95,35220.0,Short,35220.0,35120.0,35319.95,35319.95,Loss,-99.95,-4997.5
0,2021-12-27,34614.0,34375.85,Long,34614.0,34714.0,34375.85,34375.85,Loss,-238.15,-11907.5
0,2021-12-24,35194.2,35056.0,Short,35056.0,34956.0,35194.2,35194.2,Loss,-138.2,-6910.0
...,...,...,...,...,...,...,...,...,...,...,...
0,2021-08-01,32227.45,32170.5,Long,32227.45,32327.45,32170.5,32327.45,Profit,100.0,5000.0
0,2021-07-01,32214.55,32166.05,Short,32166.05,32066.05,32214.55,32214.55,Loss,-48.5,-2425.0
0,2021-06-01,31929.0,31880.0,Short,31880.0,31780.0,31929.0,31929.0,Loss,-49.0,-2450.0
0,2021-05-01,31205.0,31075.0,Long,31205.0,31305.0,31075.0,31075.0,Loss,-130.0,-6500.0


In [17]:
rpt_df

,Date,third_candle_high,third_candle_low,position,entry_price,target,stop_loss,exit_price,result,net_points,amount
0,2021-12-30,34963.9,34892.8,Long,34963.9,35063.9,34892.8,34892.8,Loss,-71.1,-3555.0
0,2021-12-29,35315.65,35238.65,Long,35315.65,35415.65,35238.65,35238.65,Loss,-77.0,-3850.0
0,2021-12-28,35319.95,35220.0,Short,35220.0,35120.0,35319.95,35319.95,Loss,-99.95,-4997.5
0,2021-12-27,34614.0,34375.85,Long,34614.0,34714.0,34375.85,34375.85,Loss,-238.15,-11907.5
0,2021-12-24,35194.2,35056.0,Short,35056.0,34956.0,35194.2,35194.2,Loss,-138.2,-6910.0
...,...,...,...,...,...,...,...,...,...,...,...
0,2021-08-01,32227.45,32170.5,Long,32227.45,32327.45,32170.5,32327.45,Profit,100.0,5000.0
0,2021-07-01,32214.55,32166.05,Short,32166.05,32066.05,32214.55,32214.55,Loss,-48.5,-2425.0
0,2021-06-01,31929.0,31880.0,Short,31880.0,31780.0,31929.0,31929.0,Loss,-49.0,-2450.0
0,2021-05-01,31205.0,31075.0,Long,31205.0,31305.0,31075.0,31075.0,Loss,-130.0,-6500.0


In [18]:
rpt_df.result.value_counts()

Loss      172
Profit     72
NA          2
Name: result, dtype: int64

In [19]:
rpt_df.amount.sum()

-463449.99999999773

In [ ]:
rpt_df

In [ ]:
pd.DataFrame([day,thrd_cndl_high,thrd_cndl_low,position,entry_price,target,stop_loss,exit_price,result]).T

In [ ]:
rpt_df.columns = ['Date','third_candle_high','third_candle_low','position',
                  'entry_price','target','stop_loss','exit_price','result']

In [ ]:
date,high,low,position,entry,targt,sl,exit_price,result